In [46]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras import backend

import time
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD

from scipy.optimize import fmin_l_bfgs_b

In [47]:
def preprocess_image(image_path):

    image=Image.open(image_path)
    image=image.resize((512,512))

    img_array=np.asarray(image,dtype='float32')
    img_array=np.expand_dims(img_array,axis=0)


    img_array[:, :, :,0]-=103.939
    img_array[:, :, :,1]-=116.779
    img_array[:, :, :,2]-=123.68
    
    img_array=img_array[:, :, :,::-1]
    return img_array

In [48]:
def deprocess_image(x):
    x = x.reshape((img_rows, img_cols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x


In [49]:
alpha= 1e-6
beta=5e-6
gamma=1e-6

In [50]:
model= VGG16(weights='imagenet',include_top=False)
layers=dict([(layer.name,layer.output) for layer in model.layers])

In [51]:
feature_extractor = keras.Model(inputs=model.inputs, outputs=layers)

In [52]:
def content_loss(content,final):
    return tf.reduce_sum(backend.square(final-content))

In [53]:
def gram_matrix(T):
    features=backend.batch_flatten(backend.permute_dimensions(T, (2,0,1)))
    gram = backend.dot(features,backend.transpose(features))
    return gram

In [54]:
def style_loss(style,final):
    sty=  gram_matrix(style)
    fin= gram_matrix(final)
    return tf.reduce_sum(backend.square(sty-fin))/ (4 * (3**2) * (262144**2))

In [55]:
def total_var_loss(T):
    a= backend.square(T[:,511,511]-T[:,1:,511,:])
    b= backend.square(T[:,511,511]-T[:,511:,1:,:])
    return tf.reduce_sum(backend.pow(a+b,1.25))

In [56]:
def tot_loss(content_image,style_image,final_image):
    
  
    input_tensor=tf.concat([content_image,style_image,final_image], axis=0)
    loss = tf.zeros(shape=())
    features = feature_extractor(input_tensor)
    
    
    layer_features=features['block2_conv2']
    content_features=layer_features[0,:,:,:]
    final_features=layer_features[2,:,:,:]

    loss=loss + alpha * content_loss(content_features,final_features)
    print(8)
    
    feature_layers=['block1_conv2','block2_conv2','block3_conv3','block4_conv3','block5_conv3']
    for layer_name in feature_layers:
        
        layer_features=features[layer_name]
        style_features=layer_features[1,:,:,:]
        final_features=layer_features[2,:,:,:]
        loss = loss + (beta/len(feature_layers))*(style_loss(style_features,final_features))
    
    loss= loss + gamma*total_var_loss(final_image)
    return loss

In [57]:

@tf.function
def compute_loss_and_grads(final_image,content_image,style_image,):
    with tf.GradientTape() as tape:
        loss = tot_loss(content_image,style_image,final_image)
    print(5)
    grads = tape.gradient(loss, final_image)
    return loss, grads

In [58]:
optimizer = SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    ))

style_image=preprocess_image('van.JFIF')
content_image=preprocess_image('effiell.jpg!d')

final_image = tf.Variable(preprocess_image('effiell.jpg!d'))
losses=[]
itera=[]


iterations = 8000
for i in range(1, iterations + 1):
    print(9)
    loss, grads = compute_loss_and_grads(
        final_image,content_image, style_image)
    optimizer.apply_gradients([(grads, final_image)])
    
    losses.append(loss)
    itera.append(i)
    print("Iteration %d: loss=%.2f" % (i, loss))
    if i == 8000:
        img = deprocess_image(final_image.numpy())


9
8
5
Iteration 1: loss=597153.75
9
Iteration 2: loss=452612.69
9
Iteration 3: loss=488118.78
9
Iteration 4: loss=12708730.00
9
Iteration 5: loss=50330923008.00
9
Iteration 6: loss=103790616307040256.00
9
Iteration 7: loss=inf
9
Iteration 8: loss=inf
9


KeyboardInterrupt: 